# World Health Organization Covid-19 Reporting

In [3]:
from bs4 import BeautifulSoup
import requests, wget, datetime, PyPDF2, os

In [12]:
site = "https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports"
req = requests.get(site)
soup = BeautifulSoup(req.content, 'html.parser')

In [13]:
sitrep = 'situation-reports' + '_' + str(datetime.date.today())
with open(sitrep, 'w') as f:
    f.writelines(req.text)

In [14]:
reports = [ 'https://www.who.int' + link for link in \
          [ l.get('href') for l in soup.find_all('a') ] \
          if link.startswith('/docs') ]

In [111]:
#verify all reports dl'd and not tmp'd or dup'd
#if reports all dl'd but for today, only get today
if not os.stat('reports'):
    os.mkdir('reports')
def dl_todays_report():
    wget.download(reports[1],out='reports/')

def dl_all_reports():
    for r in reports:
        wget.download(r,out='reports/')

# Extract table data from pdf

Date

Area | Total Cases | New Cases | Total Deaths | New Deaths | Since Last Reported Case

In [36]:
#need programatic selection of filename
#database data
#process all files
#process newest/unprocessed files


In [37]:
def pdf2text(pdf):
    page = ''
    f = open(pdf,'rb')
    pdfreader = PyPDF2.PdfFileReader(f)
    for p in range(rr.getNumPages()):
        page += pdfreader.getPage(p).extractText()
    return page
page = pdf2text('reports/20200331-sitrep-71-covid-19.pdf')

# pull date
cases

and deaths

. Data as of

In [14]:
#pull date
def get_date(page):
    dotData=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('. Data') ][0]
    day = page.splitlines()[dotData+2]
    month = page.splitlines()[dotData+4]
    year = page.splitlines()[dotData+6]
    date = "%s-%s-%s" % (month,day,year)
    return date
date = get_date(page)

# pull table data

In [38]:
#pull table data
def pull_table_data(page):
    report_begin=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('reported case') ][0] +2
    report_end=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('Numbers') ][0] 
    surveillance_data = [ l for l in page.splitlines()[report_begin:report_end] if l != ' ' ]
    return surveillance_data
surveillance_data = pull_table_data(page)

# extract by identification relativity

In [43]:
def get_data():
    result = {}
    for l in surveillance_data:
        region = ''
        area = ''
        if 'Region' in l:
            region = l
        else:
            region = region
            for c in l:
                if c.isalpha() or c.isspace():
                    area += c
            if area != '':
                block_begin = surveillance_data.index(area)
                total_confirmed = surveillance_data[block_begin+1]
                total_new = surveillance_data[block_begin+2]
                total_deaths = surveillance_data[block_begin+3]
                total_new_deaths = surveillance_data[block_begin+4]
                result[region] =  [ area,total_confirmed,total_new,total_deaths,total_new_deaths]
    return result
get_data()

ValueError: 'Lao Peoples ' is not in list

In [ ]:
for l in surveillance_data:
    region = ''
    area = ''
    trans = ''
    while l.isalpha:
        if 'Region' in l:
            region = l
        #else:
            #region = region
        
        if 'Imported' in l or 'Local' in l:
            trans = l
        else:
            for c in l:
                if c.isalpha() or c.isspace():
                    area += c           
    if area:
        block_begin = surveillance_data.index(area)
        total_confirmed = surveillance_data[block_begin+1]
        total_new = surveillance_data[block_begin+2]
        total_deaths = surveillance_data[block_begin+3]
        total_new_deaths = surveillance_data[block_begin+4]
        trans = surveillance_data[block_begin+5]
        days_since = surveillance_data[block_begin+6]
        print(region,[ area,total_confirmed,total_new,total_deaths,total_new_deaths,trans,days_since])

# extract by indexing?

In [39]:
def region_index(p):
    region_index = {}
    for ri in p:
        if 'Region' in ri:
            region_index[p.index(ri)] = ri
            #region_index.append(surveillance_data.index(ri))
    ri = [ l for l in sorted(region_index.items()) ]
    return ri
region_index(surveillance_data)

[(0, 'Western Pacific Region'),
 (140, 'European Region '),
 (572, 'East Asia Region'),
 (645, 'Eastern Mediterranean Region'),
 (798, 'Region of the Americas'),
 (1167, 'African Region')]

In [94]:
def area_block(ri):
    idx_map = {}
    for i in range(0,len(ri)-1):
        for idx in range(ri[i][0],ri[i+1][0]):
            if idx == ri[-1][0]-1:
                idx_map[ri[-1][1]] = ri[-1][0]
            #print(ri[i][1], 'range:',idx)
            idx_map[ri[i][1]] = [ i for i in range(ri[i][0],ri[i+1][0]) ] 
    return idx_map

In [95]:
area_index = area_block(region_index(surveillance_data))

In [ ]:
def f():
    r = {}
    for k,v in area_index.items():
        r[k] = v ,
    return r


In [18]:
def covid_report(pdf):
    report = {}
    page = pdf2text(pdf)
    date = get_date(page)
    data = pull_table_date(page)
    report[date] = data ,
    return report

In [48]:
print(page)

 
 
 
 
 
 
 
 
 
 
Data as reported 
by 
national authorities 
by 
10:00
 
CET
 
31 
March 2020
 
 
 
 
 
 
 
 
Coronavirus disease 2019 (COVID
-
19)
 
Situation Report 

 
71
 
 
 
HIGHLIGHTS
 
 

 
No new countries/territories/areas reported cases of COVID
-
19 in the 
past 24 
hours.
 

 
As mentioned by the 
Director
-
General in his regular media briefing yesterday
,
 
e
nsuring free movement of essential health products is vital for saving lives 
and curbing the social and economic impacts of the pandemic
. WHO is 
working intensive
ly with partners to 
increase access to life
-
saving products, 
including diagnostics, PPE, medical oxygen, ventilators and more.
 
For more 
information: 
https://www.who.int/dg/speeches/detail/who
-
director
-
general
-
s
-
opening
-
remarks
-
at
-
the
-
media
-
briefing
-
on
-
covid
-
19
---
30
-
march
-
2020
 

 
Operations support and logistics continues to monitor critical markets and in 
partnership 
with the Pandemic Supply Chain Network, ha